In [1]:
:opt no-lint

# $\lambda$-calculus with arithmetic expressions

람다계산법은 수리명제자동판별문제(Entscheidungsproblem)의 해결 불가능함을 보이기 위해
Alonzo Church가 최소한의 문법으로 고안한 이론적 소품으로써 탄생했다.
이후 프로그래밍 언어 이론의 연구에 활발히 활용되고 있으며
특히 *함수형 프로그래밍 언어*(functional programming language) 설계의 기초가 되고 있다.

이론적으로 순수한 람다계산법으로도 조건문, 자연수 계산 등 모든 것이 표현 가능하긴 하지만
순수한 람다계산법만으로 실제 컴퓨터에서 계산을 돌린다면 비효율적 수 있다.
컴퓨터 하드웨어에서 제공하는 정수 연산 등 여러 가지 다양한 기능을
람다계산법을 기초로 하여 덧붙여 나간 언어들이 바로 함수형 프로그래밍 언어이다.
또한 최근에는 함수형 프로그래밍 언어로 시작하지 않은 많은 프로그래밍 언어들이
람다계산법의 문법이나 개념을 도입하고 있기도 하므로 실무적 프로그래밍 능력 향상을 위해서도
람다계산법에 대한 기본적 이해가 있어야 한다.

이 노트북에서는 지금까지 살펴본 덧셈식과 람다계산법을 같이 함께 합쳐 놓은 언어에 대한
call-by-value 인터프리터를 작성한다.

$x,y,z,\ldots \in \textsf{Nm} = \textsf{String}$

$n \in \textsf{Int}$

$\begin{array}{lrl}
 e \in \textsf{Expr}
    &::=& x ~\mid~ \lambda x.e ~\mid e_1~e_2 \\
   &\mid& n ~\mid~ e_1 ✚\, e_2 \\
   &\mid& \textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_0
 \end{array}$

$\textsf{Env} = \textsf{Nm} \xrightarrow{\textrm{fin}} \textsf{Value}$

$\rho \in \textsf{Env} ::= \{x_1\mapsto v_1,\ldots,x_n\mapsto v_n\}$

$\textsf{Value} \subsetneq (\textsf{Expr} \times \textsf{Env}) \;\cup\; \textsf{Int}$

$v \in \textsf{Value} ::= \langle \lambda x.e, \rho \rangle ~\mid~ n$

$\displaystyle
\frac{}{\langle x,\rho\rangle \Downarrow \rho(x)} \qquad
\frac{}{\langle\lambda x.e,\rho\rangle \Downarrow \langle\lambda x.e,\rho\rangle}
$

$\displaystyle
\frac{
      e_1,\rho \Downarrow \langle\lambda x.e,\rho_1\rangle
      \quad
      e_2,\rho \Downarrow v_2 \quad e,\{x\mapsto v_2\}\rho_1 \Downarrow v
    }{e_1~e_2,\rho \Downarrow v}
$

$\displaystyle
  \frac{~~}{n,\rho \Downarrow n}
  \qquad
  \frac{e_1,\rho \Downarrow n_1 \quad e_2,\rho \Downarrow n_2}{
        e_1 ✚\, e_2,\rho \Downarrow n}(n\equiv n_1+n_2)$
        
$\displaystyle
  \frac{e,\rho \Downarrow 0 \quad e_0,\rho \Downarrow n_0}{
        \texttt{if}~ e ~\texttt{then}~ e_1 ~\texttt{else}~ e_0,\rho \Downarrow n_0}
  \qquad        
  \frac{e,\rho \Downarrow n \quad e_1,\rho \Downarrow n_1}{
        \texttt{if}~ e ~\texttt{then}~ e_1 ~\texttt{else}~ e_0 \Downarrow n_1,\rho}~(n\neq 0)$


In [2]:
-- 변수 이름은 문자열 나타낸다
type Nm = String

-- 람다식 문법 구조
data Expr = Var Nm             -- x
          | Lam Nm Expr        -- (λx.e)
          | App Expr Expr      -- (e1 e2)
          | Lit Int            -- n
          | Add Expr Expr      -- e1 + e2
          | If Expr Expr Expr  -- if e then e1 else e0
          deriving (Show, Eq)

In [3]:
-- 람다식을 보기좋게 문자열로 변환해주는 함수
ppTm (Var x) = x
ppTm (Lam x e) = "\\" ++ x ++ " -> " ++ ppTm e
ppTm (App e1 e2) = pp1 e1 ++ " " ++ pp2 e2
ppTm (Val n) = show n
ppTm (Add e1 e2) = ppp e1 ++ " + " ++ ppp e2
ppTm (If e e1 e0) = "if "++pp2 e++" then "++pp2 e1++" else "++pp2 e0 

pp1 e@(Lam{}) = paren (ppTm e)
pp1 e@(Add{}) = paren (ppTm e)
pp1 e@(If{})  = paren (ppTm e)
pp1 e         = ppTm e

pp2 e@(Var{}) = ppTm e
pp2 e@(Val{}) = ppTm e
pp2 e         = paren (ppTm e)

ppp e@(Var{}) = ppTm e
ppp e@(Val{}) = ppTm e
ppp e@(Add{}) = ppTm e
ppp e@(App{}) = ppTm e
ppp e         = paren (ppTm e)

paren s = "(" ++ s ++ ")"
brack s = "[" ++ s ++ "]"
latex s = "$" ++ s ++ "$"

-- 람다식을 보기좋게 TeX 코드로 변환해주는 함수
texTm (Var x) = x
texTm (Lam x e) = "\\lambda " ++ x ++ "." ++ texTm e
texTm (App e1 e2) = tex1 e1 ++ "~" ++ tex2 e2
texTm (Val n) = show n
texTm (Add e1 e2) = texp e1 ++ "+" ++ texp e2
texTm (If e e1 e0) = "\\texttt{if}~"++tex2 e++"~\\texttt{then}~"++tex2 e1++"~\\texttt{else}~"++tex2 e0

tex1 e@(Lam{}) = paren (texTm e)
tex1 e@(Add{}) = paren (texTm e)
tex1 e@(If{}) = paren (texTm e)
tex1 t         = texTm t

tex2 s@(Var{}) = texTm s
tex2 s@(Val{}) = texTm s
tex2 s         = paren (texTm s)

texp s@(Var{}) = texTm s
texp s@(Val{}) = texTm s
texp s@(Add{}) = texTm s
texp s@(App{}) = texTm s
texp s         = paren (texTm s)

In [4]:
idTm = Lam "x" (Var "x")
ttTm = Lam "x" (Lam "y" (Var "x")) 
ffTm = Lam "x" (Lam "y" (Var "y")) 

putStr $ ppTm idTm
putStr $ ppTm ttTm
putStr $ ppTm ffTm
putStr $ ppTm (If (Add (Val 3) (Add (Var "x") (Var "y"))) ffTm (App idTm ttTm))

\x -> x

\x -> \y -> x

\x -> \y -> y

if (3 + x + y) then (\x -> \y -> y) else ((\x -> x) (\x -> \y -> x))

In [5]:
import IHaskell.Display

html . latex $ texTm idTm
html . latex $ texTm ttTm
html . latex $ texTm ffTm
html . latex $ texTm (App (App (Var "x") (Var "y")) (Var "z"))
html . latex $ texTm (App (Var "x") (App (Var "y") (Var "z")))
html . latex $ texTm (App (App ffTm idTm) ttTm)
html . latex $ texTm (If (Add (Val 3) (Add (Var "x") (Var "y"))) ffTm (App idTm ttTm))

$\lambda x.x$

$\lambda x.\lambda y.x$

$\lambda x.\lambda y.y$

$x~y~z$

$x~(y~z)$

$(\lambda x.\lambda y.y)~(\lambda x.x)~(\lambda x.\lambda y.x)$

$\texttt{if}~(3+x+y)~\texttt{then}~(\lambda x.\lambda y.y)~\texttt{else}~((\lambda x.x)~(\lambda x.\lambda y.x))$

In [6]:
-- call-by-value evaluator
type Env = [ (Nm, Value) ]
data Value = Clos Env Tm
           | Vint Int
           deriving Show

eval :: Env -> Tm -> Value
eval env (Var x) =
    case lookup x env of
      Nothing -> error (x ++ " not defined")
      Just v  -> v
eval env e@(Lam _ _) = Clos env e
eval env (App e1 e2) =
  case v1 of
    Clos env1 (Lam x e) -> eval ((x,v2):env1) e
    _                   -> error (show v1++" not Lam")
  where
    v1 = eval env e1
    v2 = eval env e2
eval env (Val n) = Vint n
eval env (Add e1 e2) =
  case (v1, v2) of
    (Vint n1, Vint n2) -> Vint (n1 + n2)
    (Vint _ , _      ) -> error (show v2++" not int")
    _                  -> error (show v1++" not int")
  where
    v1 = eval env e1
    v2 = eval env e2
eval env (If e e1 e0) =
  case eval env e of
    Vint n -> if n==0 then eval env e0
                      else eval env e1
    _      -> error (show e++" not int")
eval env e =
    error (show e ++ " evaluation not defined yet")

In [7]:
import Data.List (intersperse)

texValue (Clos env e) = "\\langle"++texEnv env++","++texTm e++"\\rangle"
texValue (Vint n) = show n

texEnv env = "\\{"
          ++ (concat . intersperse ",")
                 [x++"\\mapsto "++texValue v | (x,v) <-env]
          ++ "\\}"

In [11]:
e1 = Lam "x" $ Lam "y" $
       If (Var "x") (Add (Var "x") (Var "y"))
                    (Add (Var "y") (Var "y"))

e2 = Val 2
e3 = Val 3

html . latex . texValue $ eval [] e1
html . latex . texValue $ eval [] e2
html . latex . texValue $ eval [] e3

$\langle\{\},\lambda x.\lambda y.\texttt{if}~x~\texttt{then}~(x+y)~\texttt{else}~(y+y)\rangle$

$2$

$3$

In [12]:
html . latex $ texTm (App e1 e2)
html . latex . texValue $ eval [] (App e1 e2)

$(\lambda x.\lambda y.\texttt{if}~x~\texttt{then}~(x+y)~\texttt{else}~(y+y))~2$

$\langle\{x\mapsto 2\},\lambda y.\texttt{if}~x~\texttt{then}~(x+y)~\texttt{else}~(y+y)\rangle$

In [13]:
html . latex $ texTm (App (App e1 e2) e3)
html . latex . texValue $ eval [] (App (App e1 e2) e3)

$(\lambda x.\lambda y.\texttt{if}~x~\texttt{then}~(x+y)~\texttt{else}~(y+y))~2~3$

$5$

In [14]:
html . latex $ texTm (App (App e1 (Val 0)) e3)
html . latex . texValue $ eval [] (App (App e1 (Val 0)) e3)

$(\lambda x.\lambda y.\texttt{if}~x~\texttt{then}~(x+y)~\texttt{else}~(y+y))~0~3$

$6$

---
## Recursion using a fixpoint combinator

$f(x)=x$를 만족하는 $x$를 함수 $f$의 fixpoint라고 말한다.

fixpoint combinator란 람다계산법에서 임의의 함수 $f$에 대해 $f$의 fixpoint를 찾아주는 역할을 하는 고차함수라고 생각하면 된다.

$Z$가 fixpoint combinator라는 말은 $Z(f)$가 $f$의 fixpoint 즉 $f(Z(f)) = Z(f)$라는 뜻이다.

람다계산법에서는 함수 적용에 괄호를 쓰지 않으므로 $f(Z~f) = Z~f$ 이렇게 표기하는 경우가 더 많을 것이다.

이것을 반대 방향으로 진행해보면 $Z~f = f(Z~f) = f(f(Z~f)) = f(f(f(Z~f))) = \cdots$
이렇게 함수 $f$를 몇번이고 반복 적용하는 계산을 표현할 수 있다.

In [15]:
-- Z = λf.(λx.f(λz.xxz))(λx.f(λz.xxz))
cZ = Lam "f" (App e e)
    where
        e = Lam "x" $ f `App` Lam "z" (App x x `App` z)
        f = Var "f"
        x = Var "x"
        z = Var "z"

html . latex . texTm $ cZ

$\lambda f.(\lambda x.f~(\lambda z.x~x~z))~(\lambda x.f~(\lambda z.x~x~z))$

In [18]:
-- λf.λi.if i then i + f(i + -1) else i
e4 = Lam "f" . Lam "i" $ If i (i `Add` App f (i `Add` Val (-1))) i          
    where
        f = Var "f"
        i = Var "i"

html . latex . texTm $ e4

$\lambda f.\lambda i.\texttt{if}~i~\texttt{then}~(i+f~(i+-1))~\texttt{else}~i$

In [19]:
-- Z (λf.λi.if i then i + f(i + -1) else i)  100
eval [] (App cZ e4 `App` Val 100) -- 100부터 0까지의 총합

Vint 5050

100은 너무 길어지니까 2정도로만 람다식을 손으로 전개해 보자.

```
cZ e4 2
= (λf.(λx.f(λz.xxz)) (λx.f(λz.xxz))) e4 2
= (λx.e4(λz.xxz)) (λx.e4(λz.xxz)) 2        여기서 w = (λx.e4(λz.xxz))라고 하자
= (λx.e4(λz.xxz)) w 2
= e4 (λz.w w z) 2
= (λf.λi.if i then i + f (i + -1) else i) (λz.w w z) 2
= (λi.if i then i + (λz.w w z) (i + -1) else i) 2
= if 2 then 2 + (λz.w w z) (2 + -1) else 2
= 2 + (λz.w w z) 1
= 2 + w w 1
= 2 + (λx.e4(λz.xxz)) w 1
= 2 + e4 (λz.w w z) 1
= 2 + (λf.λi.if i then i + f (i + -1) else i) (λz.w w z) 1
= 2 + (λi.if i then i + (λz.w w z) (i + -1) else i) 1
= 2 + if 1 then 1 + (λz.w w z) (1 + -1) else 1
= 2 + 1 + (λz.w w z) (1 + -1)
= 2 + 1 + w w 0
= 2 + 1 + (λx.e4(λz.xxz)) w 0
= 2 + 1 + e4 (λz.w w z) 0
= 2 + 1 + (λf.λi.if i then i + f (i + -1) else i) (λz.w w z) 0
= ...
= 2 + 1 + if 0 then 0 + f (0 + -1) else 0
= 2 + 1 + 0
```


참고로 $Y=\lambda f.(\lambda x.f(xx))(\lambda x.f(xx))$라는 $Z$보다 조금 더 간단한 fixpoint combinator가 있다.
그런데 call-by-value 또는 applicative-order 계산 방식에서는 이 $Y$-combinator는 계산을 끝내지 못하고 무한히 진행되기만 한다.
$Z$-combinator는 그래서 call-by-value 또는 applicative-order 에서도 재귀적 계산을 표현하는 데 활용할 수 있다.
$Y$나 $Z$ 말고도 람다계산법에서 fixpoint-combinator들이 더 존재한다.